# plan

# 1

## import

In [59]:
from typing import Self
from dataclasses import dataclass

from lib_scrap import aaa

## funcs

In [60]:
Price = int
Notional = Price
Size = int

@dataclass
class IOrder:
    price: Price
    size: Size

class IBook:
    def match(self: Self, order: IOrder):
        ...
        
    def aaa():
        ...

In [61]:
class AmmOrder(IOrder):
    ...

@dataclass
class AmmBook(IBook):
    mid: Price
    bins: dict[Price, Size]
    
    def match(self: Self, order: AmmOrder):
        ...

In [62]:
class CustomOrder(IOrder):
    ...

@dataclass
class CustomBook:
    mid: Price
    bins: dict[Price, Size]
    
def match(book: CustomBook, order: CustomOrder):
    ...

## sandbox

In [219]:
@dataclass
class ABook:
    index: int
    book: list[Size]
    price_tick: Price

    def __post_init__(self: Self):
        self.book_len: int = len(self.book)
    
    def price(self: Self, index: int) -> Price:
        return self.price_tick * index

    def size(self: Self, index: int) -> Size:
        return self.book[index] if 0 <= index < self.book_len else 0

    def best_index(self: Self, is_bid: bool) -> int:
        return self.index - 1 if is_bid else self.index

    def best_order(self: Self, is_bid: bool) -> (Price, Size):
        index = self.best_index(is_bid)
        return self.price(index), self.size(index)

    # mutates
    def match(self: Self, price: Price, size: Size) -> (Notional, Size):
        # if size >> maker_size: wipe & continue else: hit & stop
        # size = 3; maker_size = -5 -> -2. stop
        # size = 7; maker_size = -5 -> 2. cont
        
        # maker_size = 2; new_maker_size = -1, 0, 1; c s s; f t t
        # maker_size = 0; new_maker_size = -1, 0, 1; c s c; f t f
        # maker_size = -3; new_maker_size = -2, 0, 2; s s c; t t f

        is_bid_maker = size < 0  # maker is bid (opposite of taker)
        maker_price, maker_size = self.best_order(is_bid_maker)
        new_maker_size = maker_size + size
        
        if (price != maker_price and is_bid_maker == (maker_price < price)):
            # cease: no more match
            print(f'cease: {0} @ {maker_price}')
            return 0, 0
        
        if (maker_size != 0) and ((new_maker_size == 0) or ((maker_size > 0) == (new_maker_size > 0))):
            # stop
            print(f'stop: {size} @ {maker_price}')
            self.book[self.best_index(not is_bid_maker)] += size
            self.book[self.best_index(is_bid_maker)] += size
            return maker_price * size, size
        else:
            # cont
            match_size = self.book[self.best_index(is_bid_maker)]
            print(f'cont: {match_size} @ {maker_price}')
            self.book[self.best_index(not is_bid_maker)] -= match_size
            self.book[self.best_index(is_bid_maker)] = 0
            self.index += -1 if is_bid_maker else 1
            if 0 < self.index < self.book_len:
                print(self.index)
                next_notional, next_size = self.match(price, size - match_size)
            else:
                # end of book
                print(f'end: {0} @ {maker_price}')
                self.index = min(max(self.index, 1), self.book_len - 1)
                next_notional, next_size = 0, 0
            return maker_price * match_size + next_notional, match_size + next_size

In [220]:
empty_book = ABook(1, [0 for i in range(6)], 20)
some_book = ABook(3, [1, 5, 3, -2, 0, -2], 20)

In [221]:
empty_book, some_book

(ABook(index=1, book=[0, 0, 0, 0, 0, 0], price_tick=20),
 ABook(index=3, book=[1, 5, 3, -2, 0, -2], price_tick=20))

In [222]:
empty_book.best_order(True), empty_book.best_order(False)

((0, 0), (20, 0))

In [223]:
some_book.best_order(True), some_book.best_order(False)

((40, 3), (60, -2))

In [224]:
empty_book.match(200, 20), empty_book

cont: 0 @ 20
2
cont: 0 @ 40
3
cont: 0 @ 60
4
cont: 0 @ 80
5
cont: 0 @ 100
end: 0 @ 100


((0, 0), ABook(index=5, book=[0, 0, 0, 0, 0, 0], price_tick=20))

In [225]:
empty_book.match(-1, -20), empty_book

cont: 0 @ 80
4
cont: 0 @ 60
3
cont: 0 @ 40
2
cont: 0 @ 20
1
cont: 0 @ 0
end: 0 @ 0


((0, 0), ABook(index=1, book=[0, 0, 0, 0, 0, 0], price_tick=20))

In [226]:
some_book.match(200, 20), some_book

cont: -2 @ 60
4
cont: 0 @ 80
5
cont: -2 @ 100
end: 0 @ 100


((-320, -4), ABook(index=5, book=[1, 5, 5, 0, 2, 0], price_tick=20))

In [227]:
some_book.match(-1, -20), some_book

cont: 2 @ 80
4
cont: 0 @ 60
3
cont: 5 @ 40
2
cont: 5 @ 20
1
cont: 1 @ 0
end: 0 @ 0


((460, 13), ABook(index=1, book=[0, -1, -5, -5, 0, -2], price_tick=20))